# BENCHMARKING Y PROFILING
Temas que se cubren en el notebook:

* Diseño de la aplicación
* Escritura de tests y benchmarks
* Mejora de los tests y benchmarks con pytest-benchmark
* Localización de cuellos de botella con cProfile
* Optimizando el código
* Usando el módulo dis
* Profiling del uso de la memoria con memory_profiler


## Diseñando una aplicación
Principios de la optimización del código:

Que funcione > Que esté bien hecho > Que sea rápido

### Diseño de un simulador de partículas
Para demostrar/aplicar los principios anteriores voy a utilizar un simulador de partículas.
Me parece un poco presuntuoso el normbre. Más bien son unas bolas dando vueltas al rededor de un punto fijo.

Cada partícula tiene unas coordenadas $(x,y)$ y una velocidad angular $\vec{V}$, que hay descomponer en sus respectivas velocidades $(\vec{V}_x,\vec{V}_y)$.

Bueno he modificado un poco las funciones originales porque sino la animación no aparece en el notebook, ni al pasarla a html. Lo demás es igual.


In [47]:
# generic particle class. It stores the particle's position x y and it's angular veocity
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel

In [64]:
# this class will encapsulate the laws of motion and will be responsible for changing the positions of the particles over time
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
    
    def evolve(self,totalTime):
        dt = 0.00001
        nsteps = int(totalTime/dt)

        for i in range(nsteps):
            for p in self.particles:

                # 1. calculate de direction
                norm = (p.x**2 + p.y**2)**0.5
                Vx = -(p.y*p.ang_vel)/norm
                Vy = (p.x*p.ang_vel)/ norm
                
                # 2. calculate the displacement
                dx = dt * Vx; p.x += dx
                dy = dt * Vy; p.y += dy

                # 3. repeat for all the time steps

In [48]:
# Función de animación
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
plt.ioff()
rc('animation', html='jshtml')
def visualize(simulator):
    X = [p.x for p in simulator.particles]
    Y = [p.y for p in simulator.particles]

    fig = plt.figure()
    ax = plt.subplot(111, aspect='equal')
    line, = ax.plot(X, Y, 'ro')

    # Axis limits
    plt.xlim(-0.75, 0.75)
    plt.ylim(-0.75, 0.75)

    # It will be run when the animation starts
    def init():
        line.set_data([], [])
        return line, # The comma is important!
    
    def animate(i):
    # We let the particle evolve for 0.01 time units
        simulator.evolve(0.01)
        X = [p.x for p in simulator.particles]
        Y = [p.y for p in simulator.particles]
        line.set_data(X, Y)
        return line,
    
    # Call the animate function each 10 ms
    anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True, interval=10)
    return anim 

In [49]:
# declaración de 3 partículas y un objeto simulador con ellas
particles = [
    Particle(0.3, 0.5, 1),
    Particle(0.0, -0.5, -1),
    Particle(-0.1, -0.4, 3)
    ]
simulador = ParticleSimulator(particles)

In [50]:
animacion = visualize(simulador)
animacion

## Escritura de tests y benchmarks

Ahora que tenemos nuestro simulador, podemos empezar a medir su desempeño para que el simulador pueda albergar la máxima cantidad de partículas posible.

Necesitamos un test para saber que los resultados de la simulación son correctos. Mas específicamente, lo que vamos a hacer ahora es implentar *unit tests*, cuyo objetivo el de verificar la lógica prevista para el programa, independientemente de los detalles de la implementación, los cuales pueden cambiar durante la operación.

Un conjunto de pruebas sólido garantiza que la implementación sea correcta en cada iteración para que podamos experimentar distintas cosas con el código, con la confianza de que, si el conjunto de pruebas pasa, el código seguirá funcionando como se espera.

El test cogerá 3 partículas, las simulará durante 0.1 unidades de tiempo y comparará los
resultados con los de una implementación de referencia.Una buena manera de organizar los tests es usar una función separada para cada aspecto diferente (o unidad) de la aplicación.


```{margin}
Recordémos de las clases de programación científica que el **eps** es el error máximo que puede tener una máquina a la hora de representar dos números "iguales".
```

In [65]:
def test_evolve():
    particles = [Particle( 0.3, 0.5, +1),
                 Particle( 0.0, -0.5, -1),
                 Particle(-0.1, -0.4, +3)
        ]

    simulator = ParticleSimulator(particles)
    simulator.evolve(0.1)

    p0, p1, p2 = particles
    
    def fequal(a, b, eps=1e-5):
        return abs(a - b) < eps
        
    assert fequal(p0.x, 0.210269)
    assert fequal(p0.y, 0.543863)
    assert fequal(p1.x, -0.099334)
    assert fequal(p1.y, -0.490034)
    assert fequal(p2.x, 0.191358)
    assert fequal(p2.y, -0.365227)

La declaración *assert** lanzará un error si las condiciones no se cumplen. Al ejecutar la función, si no se observa ningún resultado o no se imprime nada, es que las condiciones se cumplen y esta todo ok.

In [66]:
test_evolve()

Todo correcto.

Un test asegura que la funcionalidad funciona, pero no dice cuanto tarda. Un **benchmark** es un caso de uso sencillo y representativo que nos dice el tiempo de ejecución de una aplicación. Son útiles para saber como de rápido es nuestro programa con cada nueva versión que implementamos.

Para el siguiente benchmark vamos a instanciar 1000 partículas con atributos generados aleatoriamente y las vamos a simular durante 0.1 segs, a ver que pasa.

In [67]:
def benchmark():
    from random import uniform

    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0))
                for i in range(1000)]
                
    simulator = ParticleSimulator(particles)
    simulator.evolve(0.1)

In [68]:
benchmark()